# MNIST CNN

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("data/", one_hot = True)

W0720 06:33:30.942200 139953958422400 deprecation.py:323] From <ipython-input-3-951f62e69444>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0720 06:33:30.948192 139953958422400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0720 06:33:30.950064 139953958422400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/train-images-idx3-ubyte.gz


W0720 06:33:31.368438 139953958422400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0720 06:33:31.374970 139953958422400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0720 06:33:31.465356 139953958422400 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be remove

Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz


# Helper

## Init Weights


In [0]:
def init_weights(weight_shape):
  init_random_dist = tf.truncated_normal(weight_shape, stddev = 0.1)
  return tf.Variable(init_random_dist)

## Init Bias


In [0]:
def init_bias(bias_shape):
  init_bias_vals = tf.constant(0.1, shape = bias_shape)
  return tf.Variable(init_bias_vals)

## Conv2D

In [0]:
def conv2d(x, W):
  # x --> [batch, H, W, Channels]
  # W --> [filter H, filter W, Channels IN, Channels OUT]
  
  return tf.nn.conv2d(x,
                      W,
                      strides = [1, 1, 1, 1],
                      padding = 'SAME')

## Pooling

In [0]:
def max_pool_2by2(x):
  # x --> [batch, H, W, Channels]
  return tf.nn.max_pool(x,
                        ksize = [1, 2, 2, 1],
                        strides= [1, 2, 2, 1],
                        padding = 'SAME')

## Convolutional Layer

In [0]:
def convolutional_layer(input_x, shape):
  W = init_weights(shape)
  b = init_bias([shape[3]])
  return tf.nn.relu(conv2d(input_x, W) + b)

## Normal (Fully Connected) Layer

In [0]:
def normal_full_layer(input_layer, size):
  input_size = int(input_layer.get_shape()[1])
  W = init_weights([input_size, size])
  b = init_bias([size])
  return tf.matmul(input_layer, W) + b

In [0]:
# Placeholder
x = tf.placeholder(tf.float32, shape = [None, 784])

In [0]:
y_true = tf.placeholder(tf.float32, shape = [None, 10])

In [0]:
# Layers
x_image = tf.reshape(x, [-1, 28,28, 1])

In [0]:
convo_1 = convolutional_layer(x_image, shape = [5, 5, 1, 32])
convo_1_pooling = max_pool_2by2(convo_1)

In [0]:
convo_2 = convolutional_layer(convo_1_pooling, shape = [5, 5, 32, 64])
convo_2_pooling = max_pool_2by2(convo_2)

In [0]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7*7*64])
full_layer_1 = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [16]:
# Dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_1, keep_prob = hold_prob)

W0720 06:33:31.948602 139953958422400 deprecation.py:506] From <ipython-input-16-4573cd35ed11>:2: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [18]:
# Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true,
                                                                        logits = y_pred))

W0720 06:33:32.000020 139953958422400 deprecation.py:323] From <ipython-input-18-84f6bce27b00>:2: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train = optimizer.minimize(cross_entropy)

In [0]:
init = tf.global_variables_initializer()

In [22]:
steps = 5000

with tf.Session() as sess:
  sess.run(init)
  
  for i in range(steps):
    batch_x, batch_y = mnist.train.next_batch(50)
    
    sess.run(train,
             feed_dict = {x: batch_x,
                          y_true: batch_y,
                          hold_prob: 0.5})
    
    if i%100 == 0:
      
      print("ON STEP : {}".format(i))
      print(">>> Accuracy: ")
      matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
      
      acc = tf.reduce_mean(tf.cast(matches, tf.float32))
      print(sess.run(acc,
                     feed_dict = {x: mnist.test.images,
                                  y_true: mnist.test.labels,
                                  hold_prob: 1.0}))
      print('\n')
      

ON STEP : 0
>>> Accuracy: 
0.0973


ON STEP : 100
>>> Accuracy: 
0.9416


ON STEP : 200
>>> Accuracy: 
0.9609


ON STEP : 300
>>> Accuracy: 
0.9714


ON STEP : 400
>>> Accuracy: 
0.9749


ON STEP : 500
>>> Accuracy: 
0.9732


ON STEP : 600
>>> Accuracy: 
0.9761


ON STEP : 700
>>> Accuracy: 
0.98


ON STEP : 800
>>> Accuracy: 
0.9772


ON STEP : 900
>>> Accuracy: 
0.9817


ON STEP : 1000
>>> Accuracy: 
0.9822


ON STEP : 1100
>>> Accuracy: 
0.9794


ON STEP : 1200
>>> Accuracy: 
0.9855


ON STEP : 1300
>>> Accuracy: 
0.987


ON STEP : 1400
>>> Accuracy: 
0.9839


ON STEP : 1500
>>> Accuracy: 
0.9869


ON STEP : 1600
>>> Accuracy: 
0.9877


ON STEP : 1700
>>> Accuracy: 
0.9883


ON STEP : 1800
>>> Accuracy: 
0.9853


ON STEP : 1900
>>> Accuracy: 
0.9883


ON STEP : 2000
>>> Accuracy: 
0.9858


ON STEP : 2100
>>> Accuracy: 
0.9874


ON STEP : 2200
>>> Accuracy: 
0.988


ON STEP : 2300
>>> Accuracy: 
0.9878


ON STEP : 2400
>>> Accuracy: 
0.9879


ON STEP : 2500
>>> Accuracy: 
0.9891


ON